# Tutorial: Component Design
## Step 2/2: Simulate the MMI design

Now, we will simulate the MMI design we have created and try to optimize its transmission:

<img src="_image/MMI_simulated.png" />


First, we import some mathematical, plotting libraries as usual, and also the previously defined MMI layout.

In [ ]:
%matplotlib inline
from technologies import silicon_photonics
import pylab as plt
import numpy as np
import sys

plt.rcParams['figure.figsize'] = (12, 12)
sys.path.insert(0, '.')

from support_files.mmi_layout import SimpleMMI
mmi = SimpleMMI()
mmi_layout = mmi.Layout(length=30)
mmi_layout.visualize(annotate=True)

Luceda predefined an algorithm to simulate this splitter using **CAMFR**. CAMFR is the built-in 1D mode solver and 2D mode propagation tool of IPKISS.It is in the `support_files.simulate_splitter` package:

In [ ]:
from support_files.simulate_splitter import simulate_splitter

Now we can simulate the splitter and plot the field. 

>**Note** 
>* Since CAMFR is a 2D mode propagation tool, the effective index method is used to approximate the structure. The effective indices of the material stacks are set in the technology file (`technologies.silicon_photonics`). The device is simulated in TM mode (H-field out-of-plane) such as to emulate a TE mode in full vectorial simulation. 
>* In order to exploit the symmetry of this device, we put an "Electrical wall" in the horizontal center line and simulate only the upper half of the splitter. The Electrical wall will ensure that the E-field is perpendicular to this wall (in plane) as is the case for a TM mode in this 2D simulation.

In [ ]:
mmi = SimpleMMI()
mmi_layout = mmi.Layout(length=60)

transmission, reflection = simulate_splitter(layout=mmi_layout, wavelength=1.50, north=3.0, aspect=3.0)
print(transmission, reflection)

We made the MMI very long on purpose, in order to see the optical behaviour.
At around x=39um, we can see the first 1:1 self-imaging point. At around x=22um an x=57um we can see the 1:2 imaging points.

### Experiment with the length and output waveguide spacing

<font color="red"> Exercise: Run the following code blocks, then play with the interactive simulation to see how parameters **'length'** (MMI body length) and **'spacing'** (waveguide spacing) affect the device behavior.</font>

In [ ]:
from ipywidgets import interact

In [ ]:
def simulate_mmi(length=22.0, spacing=2.0, plot=True):
    mmi = SimpleMMI()
    mmi_layout = mmi.Layout(length=length, 
                            waveguide_spacing=spacing)
    transmission, reflection = simulate_splitter(layout=mmi_layout, wavelength=1.5, north=3.0, plot=plot)
    return transmission, reflection

In [ ]:
interact(simulate_mmi, length=(10, 40, 1), spacing=(1.5, 2.5, 0.1))

### Optimize the transmission

The `simulate_splitter` function also returns the transmission and reflection simulated by CAMFR:

In [ ]:
transmission, reflection = simulate_mmi(length=14.0, spacing=2.0, plot=False)

In [ ]:
print("transmission: {}, reflection:{}".format(transmission, reflection))

We can loop over the parameter space (length, spacing) and plot the transmission. 

In the code below, we can observe the following:

* First, we create the axes of the parameter space (length, spacing) using `np.linspace`, which gives a linear space between a start value and end value, with a given number of points.
* Secondly, a 2D matrix is created to store the result. This matrix is filled with 0 initially (`np.zeros`), and each dimension the same size as one of the parameter spaces `(len(lengths), len(spacings))`
* A nested for loop (two for loops inside each other) is then used to run the simulation for each combination of `length` and `spacing`, and store the result in the result matrix.
* The `enumerate` function is used to obtain an index into the result matrix as well as the actual value of the parameter. For instance, in each step of the for loop, `enumerate(lengths)` returns the position as well as the value of a length in the `lengths` array.
* In total, 15 x 5 simulations are run.

In [ ]:
lengths = np.linspace(13.0, 15.0, 15)
spacings = np.linspace(1.8, 2.4, 5)

transmission = np.zeros(shape=(len(lengths), len(spacings)))

for length_pos, length in enumerate(lengths):
    for spacing_pos, spacing in enumerate(spacings):
        transmission[length_pos, spacing_pos] = 20*np.log10(simulate_mmi(length=length, spacing=spacing, plot=False)[0]) #Transmission in db
        print("simulating length: {}, spacing: {} - transmission {} ".format(length, spacing, transmission[length_pos, spacing_pos]))
print("done")

Finally, we can plot the simulated transmission over the parameter space. We can use the python plotting functions for that. `pcolormesh` creates a colored 2D plot. Each axis is a parameter (spacing, width) and the colors represent the transmission (note: we need to do some preformatting on the data so the axis plot properly). Using `colorbar` we add a scale to the figure.

In [ ]:
dx = spacings[1] - spacings[0]
dy = lengths[1] - lengths[0]
x2, y2 = np.meshgrid(np.arange(spacings[0],spacings[-1] + dx, dx) - dx / 2.,
                     np.arange(lengths[0],lengths[-1] + dy, dy) - dy / 2)
plt.pcolormesh(x2, y2, transmission)
plt.colorbar()
plt.xlabel("waveguide_spacing")
plt.ylabel("length")
plt.axis([x2.min(), x2.max(), y2.min(), y2.max()])
plt.xticks(spacings)
plt.yticks(lengths)
plt.title("MMI transmission as a function of length and waveguide spacing")

<font color="red">

##### Excercise

From the plot above, pick the parameter values to simulate the final MMI.

In [ ]:
# add your code here
